In [516]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")

In [541]:
elo_data = pd.read_sql_query(
    "Select DISTINCT * From Elo_AllMatches",
    con=devengine,)


Wall time: 140 ms


In [518]:
elo_data['Elo_Diff']=elo_data['Elo_Fav_Elo']-elo_data['Elo_Dog_Elo']
elo_data=elo_data[((elo_data['Elo_Diff']).lt(100))!=True]
elo_data['Winner']=elo_data.apply(lambda x: 'Elo_Fav' if x['Elo_Fav']==x['Winner'] else 'Elo_Dog',axis=1)
#elo_data['Winner_Odds']=elo_data.apply(lambda x: x['Elo_Fav_Odds'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Odds'],axis=1)
#elo_data['Winner_Est_Odds']=elo_data.apply(lambda x: x['Elo_Fav_Est_Odds'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Est_Odds'],axis=1)
#elo_data['Winner_Elo']=elo_data.apply(lambda x: x['Elo_Fav_Elo'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Elo'],axis=1)
train_data=elo_data[elo_data['Date']<'2022-09-01']
test_data=elo_data[elo_data['Date']>='2022-09-01']
test_data_resulted=test_data.reset_index()
test_data=test_data.drop(columns=['Surface','Date','Sex','Elo_Fav','Elo_Dog'])
train_data=train_data.drop(columns=['Surface','Date','Sex','Elo_Fav','Elo_Dog'])


In [519]:
print(len(test_data))
print(len(test_data_resulted))

792
792


In [520]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [521]:
X = train_data.drop(["Winner"], axis=1)
y = train_data["Winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
model2 = LogisticRegression(max_iter=20000000)
#model2 = SVC()
model2.fit(X_train, y_train)
train_score2 = model2.score(X_train, y_train)
test_score2 = model2.score(X_test, y_test)
print(train_score2,test_score2)

0.7031323048153343 0.7144212523719166


In [522]:
model2.intercept_

array([-0.00952606])

In [523]:
pred2=model2.predict(test_data.drop(columns='Winner',axis=1))

In [524]:
len(test_data_resulted)

792

In [525]:
cols = [
    "Date",
    "Prediction",
    "Winner",
    "Elo_Fav",
    "Elo_Fav_Odds",
    "Elo_Dog",
    "Elo_Dog_Odds",
    # "Training_Accuracy",
    # "Testing_Accuracy",
]
df = pd.DataFrame(columns=cols)
List = []

In [526]:
for index, tuples in test_data_resulted.iterrows():
        #if index <> len(test_data_resulted):

            values = [
                test_data_resulted["Date"][index],
                pred2[index],
                test_data_resulted["Winner"][index],
                test_data_resulted["Elo_Fav"][index],
                test_data_resulted["Elo_Fav_Odds"][index],
                test_data_resulted["Elo_Dog"][index],
                test_data_resulted["Elo_Dog_Odds"][index],
                #  "{:.0%}".format(train_score2),
                #  "{:.0%}".format(test_score2),
            ]
            zipped = zip(cols, values)
            a_dictionary = dict(zipped)
            List.append(a_dictionary)

In [527]:
temp = pd.DataFrame(List)
df = pd.concat([df, temp])

In [528]:
df
df[df['Winner']==df['Prediction']]

,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds
1,2022-09-01,Elo_Fav,Elo_Fav,Ludmilla Samsonova,1.55,Leylah Fernandez,2.49
2,2022-09-01,Elo_Fav,Elo_Fav,Veronika Kudermetova,1.18,Maryna Zanevska,4.94
4,2022-09-01,Elo_Fav,Elo_Fav,Valentin Vacherot,2.06,Yosuke Watanuki,1.70
5,2022-09-01,Elo_Fav,Elo_Fav,Gabriel Decamps,1.35,Kasidit Samrej,3.06
6,2022-09-01,Elo_Fav,Elo_Fav,Shelby Rogers,1.21,Viktoria Kuzmova,4.52
...,...,...,...,...,...,...,...
786,2022-10-14,Elo_Fav,Elo_Fav,Yibing Wu,1.22,Dalibor Svrcina,4.11
787,2022-10-14,Elo_Fav,Elo_Fav,Mikael Ymer,1.45,Roberto Carballes Baena,2.75
788,2022-10-14,Elo_Fav,Elo_Fav,Felix Auger Aliassime,1.20,Oscar Otte,4.52
789,2022-10-14,Elo_Fav,Elo_Fav,Dominic Thiem,1.39,Francisco Cerundolo,2.99


In [529]:
exclude_today=df[df['Date']!='2022-10-14'].copy()

In [530]:
exclude_today['Predicton_Odds']=exclude_today.apply(lambda x: x['Elo_Fav_Odds'] if 'Elo_Fav' == x['Prediction'] else x['Elo_Dog_Odds'],axis=1)
#exclude_today['Winner_Odds']=exclude_today['Winner_Odds'].astype(float)
exclude_today_odds=exclude_today[((exclude_today['Predicton_Odds']).ge(1.9))&((exclude_today['Predicton_Odds']).le(2.5))].copy()
#exclude_today

In [531]:
len(exclude_today_odds[exclude_today_odds['Winner']==exclude_today_odds['Prediction']])/len(exclude_today_odds)

0.4375

In [532]:
today=df[df['Date']>'2022-10-13'].copy()
today['Predicton_Odds']=today.apply(lambda x: x['Elo_Fav_Odds'] if 'Elo_Fav' == x['Prediction'] else x['Elo_Dog_Odds'],axis=1)
#today=today[((today['Predicton_Odds']).le(1.8))&((today['Predicton_Odds']).ge(1))]

In [537]:
%time today

Wall time: 968 µs


,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds,Predicton_Odds
776,2022-10-14,Elo_Fav,Elo_Fav,Paula Badosa,1.20,Louisa Chirico,4.47,1.20
777,2022-10-14,Elo_Fav,Elo_Fav,Aryna Sabalenka,1.36,Sloane Stephens,3.15,1.36
778,2022-10-14,Elo_Fav,Elo_Fav,Iga Swiatek,1.34,Qinwen Zheng,3.30,1.34
779,2022-10-14,Elo_Fav,Elo_Fav,Quentin Halys,1.26,Alastair Gray,3.67,1.26
780,2022-10-14,Elo_Fav,Elo_Fav,Jurij Rodionov,1.31,Roberto Marcora,3.32,1.31
781,2022-10-14,Elo_Fav,Elo_Dog,Gregoire Barrere,1.48,Matteo Arnaldi,2.56,1.48
782,2022-10-14,Elo_Dog,Elo_Fav,Salvatore Caruso,1.88,David Ionel Nicholas,1.85,1.85
783,2022-10-14,Elo_Fav,Elo_Dog,Hugo Grenier,1.40,Mattia Bellucci,2.82,1.40
784,2022-10-14,Elo_Fav,Elo_Dog,Christopher O'Connell,1.28,Li Tu,3.52,1.28
785,2022-10-14,Elo_Fav,Elo_Fav,Kamil Majchrzak,1.22,Shintaro Mochizuki,4.00,1.22


In [534]:
len(today[today['Winner']==today['Prediction']])/len(today)

0.6875

In [535]:
today['Predicton_Odds']=today.apply(lambda x: x['Elo_Fav_Odds'] if 'Elo_Fav' == x['Prediction'] else x['Elo_Dog_Odds'],axis=1)
